### импорты и функции

In [1]:
import itertools, warnings, json, os, random
from pathlib import Path

import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import pysindy as ps
warnings.filterwarnings("ignore")

def aic(rss, k, n):        
    return n*np.log(rss/n) + 2*k

def pick_sparse(model, max_terms=7):
    """True:  число ненулевых коэф <= max_terms"""
    return (np.abs(model.coefficients()) > 1e-10).sum() <= max_terms

def inv_safe(x, eps=1e-2):
    denom = np.where(np.abs(x) > eps, x, eps)         
    return np.sign(x) / denom

def sqrt_safe(x):
    return np.sqrt(np.where(x > 0, x, 1e-2))


### Чтение данных и базовые матрицы

In [2]:

keep = ['ruonia_ruonia','roisfix_1M','roisfix_3M','roisfix_6M',
        'usd_usd_rub','eur_eur_rub','cny_cny_rub',
        'imoex_moex_close',
        'zcyc_0.5','zcyc_1.0','zcyc_5.0','zcyc_10.0']
df = pd.read_parquet(
    Path("/Users/dmlast/Documents/Projects/ruonia-forecast/data/raw/merged_20140116_20250707.parquet")
)[['KEY_DATE', *keep]].ffill().dropna()
df['KEY_DATE'] = pd.to_datetime(df['KEY_DATE'])
df.set_index('KEY_DATE', inplace=True)

r = df['ruonia_ruonia'].values.astype(float)
X_exog_df = df.drop(columns='ruonia_ruonia').astype(float)

sc_x = StandardScaler().fit(r.reshape(-1,1))
sc_u = StandardScaler().fit(X_exog_df.values)

r_z = sc_x.transform(r.reshape(-1,1)).ravel()  
U_z = sc_u.transform(X_exog_df.values)         
r_mid_z = r_z[:-1]                         

dt = 1/365
r_mid    = r_z[:-1]         
dr_z     = np.diff(r_z)     
x_train  = r_mid.reshape(-1,1)
u_train  = U_z[:-1]         


In [3]:
def base_lib(include_sin=False, degree=2):
    poly_r = ps.PolynomialLibrary(degree=degree,
                                  include_interaction=True,
                                  include_bias=True)

    id_u   = ps.IdentityLibrary()         

    gen_ru = ps.GeneralizedLibrary([poly_r, id_u])

    custom = ps.CustomLibrary(
    library_functions=[sqrt_safe, inv_safe],
    function_names=[
        lambda x, eps = 0.01: f"sqrt({x})",   
        lambda x, eps = 0.01: f"inv({x})"     
    ]
)

    libs = poly_r + id_u + gen_ru + custom
    if include_sin:
        libs += ps.FourierLibrary(1, include_bias=False)
    return libs


### Сетка гиперпараметров

In [4]:
param_grid = {
    'ofz':   [['zcyc_0.5','zcyc_1.0'], ['zcyc_5.0','zcyc_10.0']],
    'swap':  [['roisfix_1M'], ['roisfix_3M','roisfix_6M']],
    'fx':    ['off','abs','full'],      
    'optimizer': ['stlsq','sr3'],
    'alpha': [1e-3,1e-2, 0.5, 0.1],
    'thr':   [0.01, 0.05, 0.1],
}
grid = list(itertools.product(*param_grid.values()))
best  = {'aic':np.inf}


In [5]:
import warnings
from sklearn.metrics import mean_squared_error
from numpy.linalg import LinAlgError

best = {
    'aic':      np.inf,
    'cfg':      None,
    'model_mu': None,
    'sel':      None,
    'cols':     None,   
}

for ofz, swap, fx_mode, opt_name, alpha, thr in grid:
    cols = swap + ofz + ['imoex_moex_close']
    if fx_mode != 'off':
        cols += ['usd_usd_rub', 'eur_eur_rub', 'cny_cny_rub']

    sel = [list(X_exog_df.columns).index(c) for c in cols]
    u_g = u_train[:, sel]

    lib = base_lib(include_sin=False, degree=2)
    if opt_name == 'stlsq':
        opt = ps.STLSQ(alpha=alpha, threshold=thr)
    else:
        if alpha == 0:
            continue
        nu  = 1.0 / alpha
        opt = ps.SR3(threshold=thr, nu=nu, max_iter=1000, tol=1e-6)

    model_mu = ps.SINDy(
        feature_library=lib,
        optimizer=opt,
        feature_names=['r_z'] + cols
    )

    try:
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore')
            model_mu.fit(x_train, t=dt, x_dot=dr_z, u=u_g)
    except (LinAlgError, ValueError):
        continue

    coefs = model_mu.coefficients()[0]
    nnz   = np.sum(np.abs(coefs) > 1e-8)
    if not (3 <= nnz <= 7):
        continue

    pred = model_mu.predict(x_train, u=u_g)[:, 0]
    rss  = mean_squared_error(dr_z, pred) * len(dr_z)
    aic  = len(dr_z) * np.log(rss / len(dr_z)) + 2 * nnz

    if aic < best['aic']:
        best.update(
            aic      = aic,
            model_mu = model_mu,
            sel      = sel,
            cfg      = (ofz, swap, fx_mode, opt_name, alpha, thr),
            cols     = cols,             
        )

print(f" Лучший набор cols: {best['cols']}")
print(f"  Параметры grid: {best['cfg']},  AIC = {best['aic']:.1f}")
best['model_mu'].print(precision=3)



/opt/anaconda3/lib/python3.11/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1)

 Лучший набор cols: ['roisfix_1M', 'zcyc_5.0', 'zcyc_10.0', 'imoex_moex_close', 'usd_usd_rub', 'eur_eur_rub', 'cny_cny_rub']
  Параметры grid: (['zcyc_5.0', 'zcyc_10.0'], ['roisfix_1M'], 'abs', 'stlsq', 0.5, 0.05),  AIC = -22846.4
(r_z)' = 0.071 r_z^2 + -0.131 r_z roisfix_1M + 0.059 roisfix_1M^2 + 0.071 r_z^2 + -0.131 r_z roisfix_1M + 0.059 roisfix_1M^2


### Модель \sigma^2: квадраты приращений

_В процессе_